In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
import json  # Para carregar os rótulos

np.set_printoptions(suppress=True)

# Carregar a estrutura do modelo
with open('modelo_classificacao_pessoas.json', 'r') as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)

# Carregar os pesos do modelo
model.load_weights('modelo_classificacao_pessoas.weights.h5')

# Carregar os rótulos
with open("rotulos.txt", "r") as txt_file:
    class_names = [line.split(":")[0] for line in txt_file.readlines()]

# Carregar o classificador Haar Cascade para detecção de faces
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Iniciar a captura de vídeo
camera = cv2.VideoCapture(1)

while True:
    # Capturar a imagem da webcam
    ret, image = camera.read()

    if not ret:
        print("Falha ao capturar a imagem da webcam.")
        break

    # Converter a imagem para escala de cinza (necessário para o Haar Cascade)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detectar faces
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Desenhar um retângulo ao redor de cada face detectada
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Redimensionar a face detectada para o tamanho esperado pelo modelo
        face_resized = cv2.resize(image[y:y+h, x:x+w], (128, 128), interpolation=cv2.INTER_AREA)

        # Converter a face para um array numpy e expandir as dimensões
        face_array = np.asarray(face_resized, dtype=np.float32).reshape(1, 128, 128, 3)

        # Normalizar a imagem
        face_array /= 255.0

        # Fazer a previsão
        prediction = model.predict(face_array)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Adicionar o nome da classe e a pontuação de confiança na imagem
        cv2.putText(image, f'{class_name} ({np.round(confidence_score * 100, 2)}%)', (x, y-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Mostrar a imagem em uma janela com as faces detectadas e identificadas
    cv2.imshow("Webcam Image", image)

    # Ouvir por teclas pressionadas
    keyboard_input = cv2.waitKey(1)

    # 27 é o ASCII para a tecla 'esc' no teclado
    if keyboard_input == 27:
        break

# Liberar a câmera e fechar janelas
camera.release()
cv2.destroyAllWindows()

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 128, 128, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_5'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [2]:
!pip install keras